## Optimizing ` max_distance` parameter of `KerasVGG16PageDetector`

In [1]:
from video699.configuration import get_configuration
from video699.video.annotated import (
    AnnotatedSampledVideoScreenDetector,
    evaluate_event_detector,
    get_videos,
)
from video699.page.vgg16 import KerasVGG16PageDetector
from video699.quadrangle.rtree import RTreeDequeConvexQuadrangleTracker
from video699.event.screen import ScreenEventDetector

import numpy as np


CONFIGURATION = get_configuration()['KerasVGG16PageDetector']

Using TensorFlow backend.


In [2]:
def accuracy(max_distance):
    CONFIGURATION['max_distance'] = str(max_distance)
    num_successes_total = 0
    num_trials_total = 0
    for annotated_video in get_videos().values():
        convex_quadrangle_tracker = RTreeDequeConvexQuadrangleTracker(2)
        screen_detector = AnnotatedSampledVideoScreenDetector()
        documents = annotated_video.documents.values()
        page_detector = KerasVGG16PageDetector(documents)
        screen_event_detector = ScreenEventDetector(
            annotated_video,
            convex_quadrangle_tracker,
            screen_detector,
            page_detector
        )
        num_successes, num_trials = evaluate_event_detector(annotated_video, screen_event_detector)
        num_successes_total += num_successes
        num_trials_total += num_trials
    accuracy = 1.0 * num_successes_total / num_trials_total
    return accuracy

In [3]:
%%time
max_distances = np.linspace(0.0, 2.0, num=50)
accuracies = list(map(accuracy, max_distances))

best_accuracy, best_max_distance = max(zip(accuracies, max_distances))
print('Optimal parameters (accuracy {}):'.format(best_accuracy))
print('- max_distance: {}'.format(best_max_distance))

Optimal parameters (accuracy 0.4034229828850856):
- max_distance: 0.8979591836734693
CPU times: user 20h 19min 51s, sys: 1h 33min 15s, total: 21h 53min 6s
Wall time: 10h 27min 1s
